In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('./01_raw_data/EV_project/경제 지표/기준금리.csv', encoding='cp949')

In [13]:
df_rate = df.copy()
df_rate

,변경일자,Unnamed: 1,기준금리
0,2025,05월 29일,2.50
1,2025,02월 25일,2.75
2,2024,11월 28일,3.00
3,2024,10월 11일,3.25
4,2023,01월 13일,3.50
5,2022,11월 24일,3.25
6,2022,10월 12일,3.00
7,2022,08월 25일,2.50
8,2022,07월 13일,2.25
9,2022,05월 26일,1.75


In [14]:
import pandas as pd


df_rate['date'] = pd.to_datetime(
    df_rate['변경일자'].astype(str) + ' ' + df_rate['Unnamed: 1'],
    format='%Y %m월 %d일'
)

df_rate = df_rate[['date', '기준금리']].sort_values('date')

daily_rate = (
    df_rate
    .set_index('date')
    .resample('D')
    .ffill()
    .reset_index()
)

monthly_rate = (
    daily_rate
    .groupby(daily_rate['date'].dt.to_period('M'))
    .agg({'기준금리': 'mean'})
    .reset_index()
)
monthly_rate['년도'] = monthly_rate['date'].dt.year
monthly_rate['월']   = monthly_rate['date'].dt.month
monthly_rate = monthly_rate[['년도', '월', '기준금리']] \
    .rename(columns={'기준금리': '평균기준금리'})

monthly_rate



,년도,월,평균기준금리
0,1999,5,4.750000
1,1999,6,4.750000
2,1999,7,4.750000
3,1999,8,4.750000
4,1999,9,4.750000
...,...,...,...
308,2025,1,3.000000
309,2025,2,2.964286
310,2025,3,2.750000
311,2025,4,2.750000
